In [1]:
import pandas as pd
from lets_plot.geo_data import *
from lets_plot import *

LetsPlot.setup_html()

The geodata is provided by © OpenStreetMap contributors and is made available here under the Open Database License (ODbL).


In [2]:
LetsPlot.set(maptiles_zxy(url='https://a.tile.openstreetmap.org/{z}/{x}/{y}.png'))

In [3]:
dept = pd.read_csv("../data/usgs_dept_by_state_2019.csv")

dept.head()

,State,State Debt,Unnamed: 2,Local Debt,Unnamed: 4,State and Local Debt,Unnamed: 6,Gross State Product,Unnamed: 8,Real State Growth %,Unnamed: 10,Population (million),
0,Alabama,10.7,g,26.1,g,36.8,g,228.0,g,1.1,g,4.9,e
1,Alaska,5.9,g,3.5,g,9.4,g,55.7,g,-0.2,g,0.7,e
2,Arizona,13.3,g,30.5,g,43.8,g,361.1,g,2.3,g,7.3,e
3,Arkansas,7.3,g,14.9,g,22.2,g,131.4,g,0.5,g,3.0,e
4,California,145.0,g,298.7,g,443.7,g,3080.6,g,2.0,g,39.5,e


In [4]:
dept_gg = dept \
        .drop([" ", "Unnamed: 2", "Unnamed: 4", "State and Local Debt", 
               "Unnamed: 6", "Unnamed: 8", "Real State Growth %", "Unnamed: 10", "Population (million)"], axis=1) \
        .set_index("State") \
        .stack() \
        .reset_index() \
        .rename(columns={"level_1": "Item", 0: "$B"})


dept_gg


,State,Item,$B
0,Alabama,State Debt,10.7
1,Alabama,Local Debt,26.1
2,Alabama,Gross State Product,228.0
3,Alaska,State Debt,5.9
4,Alaska,Local Debt,3.5
...,...,...,...
148,Wisconsin,Local Debt,22.4
149,Wisconsin,Gross State Product,348.1
150,Wyoming,State Debt,0.9
151,Wyoming,Local Debt,1.4


In [5]:
ggplot(dept_gg) \
+ geom_bar(aes("State", "$B", fill="Item"), stat="identity", position="dodge", sampling="none") \
+ scale_fill_manual(values=["#FE0968", "#FF75A6", "#007BCD"]) \
+ ggsize(1000, 500)

In [6]:
states_gdf = geocode_states('us-48').get_centroids()
states_gdf.head()

,state,found name,geometry
0,Vermont,Vermont,POINT (-72.77235 43.87185)
1,Massachusetts,Massachusetts,POINT (-72.09645 42.19138)
2,New York,New York,POINT (-76.09123 42.89937)
3,Maine,Maine,POINT (-69.16085 45.26236)
4,New Hampshire,New Hampshire,POINT (-71.55179 44.00151)


In [7]:
ggplot(dept_gg) + geom_livemap() + geom_pie(aes(slice="$B", fill="Item"), size=4,
                                            stat="identity", sampling="none",
                                            map=states_gdf, map_join=("State", "state"),
                                            tooltips=layer_tooltips().title("@State").line("@Item").line("@{$B}")) + \
    scale_fill_manual(values=["#FE0968", "#FF75A6", "#007BCD"])